In [1]:
import pandas as pd

In [11]:
df = pd.read_csv("21cl.csv")
df = df.dropna(subset=['First Name'])
df

,Room,First Name,Last Name,Check In,Check Out,Room Type,KEY CODE
0,127,Kobi,Warner,6/2/2024,6/7/2024,Double,NaN
1,128,Vander,Bannister,6/2/2024,6/7/2024,Double,NaN
2,128,Jimmy,Butts,6/2/2024,6/7/2024,Double,NaN
3,129,Rishi,Byakod,6/2/2024,6/7/2024,Double,NaN
4,129,Avignesh Harish,Jagavkar,6/2/2024,6/7/2024,Double,NaN
...,...,...,...,...,...,...,...
83,414,Jaela,Parker,6/2/2024,6/7/2024,Double,NaN
84,414,Jace,Williams,6/2/2024,6/7/2024,Double,NaN
85,416,Kaila,Li,6/2/2024,6/7/2024,Double,NaN
86,416,Michelle,Hu,6/2/2024,6/7/2024,Double,NaN


In [4]:
def clean_data(df):
    # Drop rows with missing data in column: 'First Name'
    df = df.dropna(subset=['First Name'])
    # Select columns: 'First Name', 'Last Name' and 3 other columns
    df = df.loc[:, ['First Name', 'Last Name', 'Check In', 'Check Out', 'Room Type']]
    # Change column type to datetime64[ns] for columnn 'Check In' and 'Check Out'
    df['Check In'] = pd.to_datetime(df['Check In']).dt.date
    df['Check Out'] = pd.to_datetime(df['Check Out']).dt.date
    
    df = df.reset_index(drop=True)
    return df

df_clean = clean_data(df.copy())
df_clean

,First Name,Last Name,Check In,Check Out,Room Type
0,Kobi,Warner,2024-06-02,2024-06-07,Double
1,Vander,Bannister,2024-06-02,2024-06-07,Double
2,Jimmy,Butts,2024-06-02,2024-06-07,Double
3,Rishi,Byakod,2024-06-02,2024-06-07,Double
4,Avignesh Harish,Jagavkar,2024-06-02,2024-06-07,Double
...,...,...,...,...,...
83,Jaela,Parker,2024-06-02,2024-06-07,Double
84,Jace,Williams,2024-06-02,2024-06-07,Double
85,Kaila,Li,2024-06-02,2024-06-07,Double
86,Michelle,Hu,2024-06-02,2024-06-07,Double


In [180]:
df_clean['Days Stayed'] = (pd.to_datetime(df_clean['Check Out']) - pd.to_datetime(df_clean['Check In'])).dt.days

single_price = int(input("Input Single Room's Price"))
double_price = int(input("Input Double Room's Price"))
single_suite_price = single_price + 4
double_suite_price = double_price + 4

df_clean['Unit Price'] = df_clean['Room Type'].apply(lambda x: single_price if x == 'Single' else double_price if x == 'Double' else single_suite_price if x == 'Single Suite' else double_suite_price if x == 'Double Suite' else None)
# df_clean.head()

In [181]:
df_clean['Charge'] = df_clean['Days Stayed'] * df_clean['Unit Price']
# df_clean    

In [182]:
# Group the data
total_charge_per_group = df_clean.groupby(['Check In', 'Check Out', 'Room Type', 'Unit Price']).agg(
        {'Charge': 'sum', 'Days Stayed': 'sum', 'First Name': 'count'}
    ).reset_index()

    # Rename columns
total_charge_per_group.rename(columns={
        'Days Stayed': 'Item Count',
        'Unit Price': 'Unit Amount',
        'Charge': 'Charge Amount',
        'First Name': 'People'
    }, inplace=True)

    # Generate the description column
total_charge_per_group['Description'] = total_charge_per_group.apply(
        lambda row: f"{row['People']} people * ${row['Unit Amount']} {row['Room Type']} * {int(row['Item Count']/row['People'])} nights", axis=1
    )

    # Reorder columns
total_charge_per_group = total_charge_per_group[['Item Count', 'Unit Amount', 'Charge Amount', 'Check In', 'Check Out', 'Description']]

total_charge_per_group

,Item Count,Unit Amount,Charge Amount,Check In,Check Out,Description
0,4,32,128,2024-06-02,2024-06-06,1 people * $32 Double * 4 nights
1,435,32,13920,2024-06-02,2024-06-07,87 people * $32 Double * 5 nights


In [ ]:
linen = int(input("Input linen count"))